In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_excel("dataset_02052023.xlsx")

In [ ]:
data.head(20)

In [ ]:

data['Robot_ProtectiveStop'] = data['Robot_ProtectiveStop'].astype(bool)
data.head(20)


In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.dtypes

In [ ]:
 data.isnull().sum()

In [ ]:
 data.isna().sum()

In [ ]:
data = data.dropna()
print(data.isnull().sum())


In [ ]:
data.info()

In [ ]:
import numpy as np
import pandas as pd
data = pd.read_excel("dataset_02052023.xlsx")
data['Robot_ProtectiveStop'] = data['Robot_ProtectiveStop'].astype(bool)
data = data.dropna()
# Convert the DataFrame to CSV
data.to_csv('cleaned_dataset.csv', index=False)
# Download the file
from google.colab import files
files.download('cleaned_dataset.csv')


In [ ]:
def remove_rows_with_many_zeros(df, threshold=0.2):
    """Removes rows where the percentage of zeros exceeds the given threshold.

    Args:
        df: The input DataFrame.
        threshold: The maximum percentage of zeros allowed in a row (default is 0.2 or 20%).

    Returns:
        A new DataFrame with rows containing excessive zeros removed.
    """

    # Select numerical columns only for zero count calculation
    numerical_cols = df.select_dtypes(include=np.number).columns

    # Calculate the percentage of zeros in each row for numerical columns
    zero_percentage = (df[numerical_cols] == 0).sum(axis=1) / len(numerical_cols)

    # Filter out rows exceeding the threshold
    filtered_df = df[zero_percentage <= threshold]

    return filtered_df

# Apply the function to your DataFrame
data_cleaned = remove_rows_with_many_zeros(data)

# Print info to verify the change
data_cleaned.info()


In [ ]:
new_db_file = "cleaned_dataset.xlsx"

try:
    data_cleaned.to_excel(new_db_file, index=False)  # Save to Excel without the index column
    print(f"New database saved to {new_db_file}")
except Exception as e:
    print(f"An error occurred while saving the database: {e}")

In [ ]:
data_cleaned.head(10)

In [ ]:
import pandas as pd

# Convert 'Timestamp' column to datetime, handling mixed data types
try:
    data_cleaned['Timestamp'] = pd.to_datetime(data_cleaned['Timestamp'], errors='coerce')
    # Drop rows where the conversion failed (resulting in NaT values)
    data_cleaned = data_cleaned.dropna(subset=['Timestamp'])
    print("Timestamp column converted to datetime successfully.")
except KeyError:
    print("'Timestamp' column not found in the DataFrame.")
except Exception as e:
    print(f"An error occurred during conversion: {e}")


#Display the updated dataframe
data_cleaned.info()

In [ ]:
data_cleaned.head(10)

In [ ]:
# Sort the DataFrame by the 'Timestamp' column in ascending order (oldest to newest)
data_sorted = data_cleaned.sort_values(by='Timestamp')

# Display the first 20 rows of the sorted DataFrame
print(data_sorted.head(20))


sorted_db_file = "sorted_dataset.xlsx"

try:
    data_sorted.to_excel(sorted_db_file, index=False)  # Save to Excel without the index column
    print(f"Sorted database saved to {sorted_db_file}")
except Exception as e:
    print(f"An error occurred while saving the database: {e}")

In [ ]:
try:
    # Drop the 'Timestamp' column
    data_sorted = data_sorted.drop('Timestamp', axis=1)
    print("'Timestamp' column dropped successfully.")

    # Save the modified DataFrame to a new Excel file
    final_db_file = "final_dataset.xlsx"
    data_sorted.to_excel(final_db_file, index=False)
    print(f"Final database without 'Timestamp' saved to {final_db_file}")

except KeyError:
    print("'Timestamp' column not found in the DataFrame.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import pandas as pd

try:
    data = pd.read_excel("final_dataset.xlsx")
    # Delete the first column (assuming it's named 'Num')
    if 'Num' in data.columns:
        data = data.drop('Num', axis=1)
        print("'Num' column dropped successfully.")

        # Save the modified DataFrame to a new Excel file
        final_db_file = "final_dataset_no_num.xlsx"
        data.to_excel(final_db_file, index=False)
        print(f"Final database without 'Num' column saved to {final_db_file}")

    else:
        print("'Num' column not found in the DataFrame.")

except FileNotFoundError:
    print("'final_dataset.xlsx' not found. Please make sure the file exists in the current directory.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

try:
    data = pd.read_excel("final_dataset_no_num.xlsx")

    # Calculate the correlation matrix
    correlation_matrix = data.corr()

    # Display the correlation matrix
    print(correlation_matrix)

    # Visualize the correlation matrix using a heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title("Correlation Matrix of Variables")
    plt.show()

except FileNotFoundError:
    print("'final_dataset_no_num.xlsx' not found. Please make sure the file exists in the current directory.")
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
Y = data[['Robot_ProtectiveStop','grip_lost']].values
X = data.drop(columns=['Robot_ProtectiveStop','grip_lost']).values

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

try:
    data = pd.read_excel("final_dataset_no_num.xlsx")

    # Select numerical features for standardization
    numerical_features = data.select_dtypes(include=np.number).columns

    # Create a StandardScaler object
    scaler = StandardScaler()

    # Fit and transform the numerical features
    data[numerical_features] = scaler.fit_transform(data[numerical_features])

    # Save the standardized data to a new Excel file
    standardized_file = "standardized_final_dataset.xlsx"
    data.to_excel(standardized_file, index=False)
    print(f"Standardized data saved to {standardized_file}")

    # Download the file
    files.download(standardized_file)

except FileNotFoundError:
    print("'final_dataset_no_num.xlsx' not found. Please make sure the file exists in the current directory.")
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
import pandas as pd
from sklearn.utils import resample

try:
    # Load the standardized dataset
    data = pd.read_excel("/content/standardized_final_dataset.xlsx")

    # Separate majority and minority classes
    majority_class = data[data['Robot_ProtectiveStop'] == False]
    minority_class = data[data['Robot_ProtectiveStop'] == True]

    # Upsample minority class
    minority_upsampled = resample(minority_class,
                                 replace=True,     # sample with replacement
                                 n_samples=len(majority_class),    # to match majority class
                                 random_state=123) # reproducible results

    # Combine majority class with upsampled minority class
    balanced_data = pd.concat([majority_class, minority_upsampled])

    # You now have 'balanced_data' with equal class representation.
    # Save the balanced data to a new file if needed
    balanced_file = "balanced_dataset.xlsx"
    balanced_data.to_excel(balanced_file, index=False)
    print(f"Balanced data saved to {balanced_file}")


except FileNotFoundError:
    print("'standardized_final_dataset.xlsx' not found. Please make sure the file exists in the current directory.")
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from joblib import dump, load

def normalisation(X):
  scaler = MinMaxScaler()
  normalized_X = scaler.fit_transform(X)
  return normalized_X


In [ ]:
from sklearn.preprocessing import StandardScaler
def standardisation(X):
  scaler = StandardScaler()
  standardized_X = scaler.fit_transform(X)
  return standardized_X

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from joblib import dump, load

def normalisation(X):
  # Reshape to 2D for MinMaxScaler (samples, features)
  num_samples = X.shape[0]
  num_features = X.shape[1] * X.shape[2] # Total features considering seq_length
  X_2D = X.reshape(num_samples, num_features)

  scaler = MinMaxScaler()
  normalized_X_2D = scaler.fit_transform(X_2D)

  # Reshape back to 3D
  normalized_X = normalized_X_2D.reshape(X.shape)
  return normalized_X

In [ ]:
def creationSequences(X, Y, seq_length):
    seqX, seqY = [], []
    for i in range(0, len(data) - seq_length, 1):
        seqX.append(X[i:(i + seq_length)])
        seqY.append(Y[i + seq_length])
    seqX = np.array(seqX)
    seqY = np.array(seqY)
    return seqX, seqY


In [ ]:
seq_length = 10
X, y = creationSequences(X, Y, seq_length)

In [ ]:
print("Shape X:", X.shape)
print("Shape y:", y.shape)

In [ ]:
import numpy as np
print("NaNValues in X and Y")
print("Number of NaN values in X:", np.isnan(X).sum())
print("Number of NaN values in Y:", np.isnan(Y).sum())


In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest= train_test_split(X, y, test_size=0.3, random_state=0)


In [ ]:
import pandas as pd
import numpy as np

try:
    data = pd.read_excel("balanced_dataset.xlsx")
    # Convert target columns to numerical (True=1, False=0)
    data['Robot_ProtectiveStop'] = data['Robot_ProtectiveStop'].astype(int)
    data['grip_lost'] = data['grip_lost'].astype(int)

    #Verify changes
    print(data.head(25))

    #Save updated data
    final_db_file = "final_dataset_converted_targets.xlsx"
    data.to_excel(final_db_file, index=False)
    print(f"Final database with converted target columns saved to {final_db_file}")

except FileNotFoundError:
    print("'final_dataset_no_num.xlsx' not found. Please make sure the file exists in the current directory.")
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
seq_length=10

nombre_feature = 20 # Corrected number of features

# Définir Une architecture du  modèle LSTM. Les étudiants doivent tester plusieurs alternatives
model2 = Sequential()
model2.add(LSTM(units=50, return_sequences=True, input_shape=(seq_length, nombre_feature)))
model2.add(LSTM(units=50))
model2.add(Dense(units=2, activation='softmax')) # Changed to 2 units with sigmoid activation for binary classification


# Compiler le modèle
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # ils peuvent tester plusieurs algo autres que ADAM
# Entraîner le modèle avec GPU
with tf.device('/GPU:0'):
    model2.fit(Xtrain, Ytrain, epochs=10, batch_size=32, validation_data=(Xtest, Ytest))


# Évaluer le modèle
loss, accuracy = model2.evaluate(Xtest, Ytest)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from joblib import dump, load
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


seq_length=10

nombre_feature = 20 # Corrected number of features

# Définir Une architecture du  modèle LSTM. Les étudiants doivent tester plusieurs alternatives
model2 = Sequential()
model2.add(LSTM(units=50, return_sequences=True, input_shape=(seq_length, nombre_feature)))
model2.add(LSTM(units=50))
model2.add(Dense(units=2, activation='sigmoid')) # Use sigmoid for binary classification


# Compiler le modèle
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # ils peuvent tester plusieurs algo autres que ADAM

# Check for NaN values in your training data
print("NaN values in Xtrain:", np.isnan(Xtrain).any())
print("NaN values in Ytrain:", np.isnan(Ytrain).any())

# If there are NaNs, replace them with 0 (or handle them appropriately)
Xtrain = np.nan_to_num(Xtrain)
Ytrain = np.nan_to_num(Ytrain)

# Entraîner le modèle avec GPU (or CPU if you don't have a GPU)
with tf.device('/GPU:0'): # or '/CPU:0' if no GPU
    model2.fit(Xtrain, Ytrain, epochs=10, batch_size=32, validation_data=(Xtest, Ytest))


# Évaluer le modèle
loss, accuracy = model.evaluate(Xtest, Ytest)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model)

In [ ]:
model.save('model.h5')

In [ ]:
plot_model(model2)

In [ ]:
model2.save('second_model.h5')

In [ ]:
!pip install -U scikit-learn==1.3.2 scikeras==0.11.0


In [ ]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional


# Define model3 architecture
model3 = Sequential()
model3.add(Bidirectional(LSTM(units=64, return_sequences=True, input_shape=(seq_length, nombre_feature))))
model3.add(Dropout(0.2))  # Add dropout for regularization
model3.add(Bidirectional(LSTM(units=64)))
model3.add(Dropout(0.2))
model3.add(Dense(units=2, activation='sigmoid'))  # Sigmoid for binary classification

# Compile model3
model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model3 with GPU (or CPU)
with tf.device('/GPU:0'):  # Or '/CPU:0'
    model3.fit(Xtrain, Ytrain, epochs=15, batch_size=64, validation_data=(Xtest, Ytest))  # Increased epochs and batch size


# Evaluate model3
loss, accuracy = model3.evaluate(Xtest, Ytest)
print(f"Model 3 - Test Loss: {loss}")
print(f"Model 3 - Test Accuracy: {accuracy}")

plot_model(model3)
model3.save('model3.h5')


In [ ]:
plot_model(model3)

In [ ]:
model3.save('third_model.h5')

In [ ]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# Define model4 architecture using 1D CNN
model4 = Sequential()
model4.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(seq_length, nombre_feature)))
model4.add(MaxPooling1D(pool_size=2))
model4.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model4.add(MaxPooling1D(pool_size=2))
model4.add(Flatten())
model4.add(Dense(100, activation='relu')) # Added a dense layer
model4.add(Dense(units=2, activation='softmax'))  # Sigmoid for binary classification

# Compile model4
model4.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model4 with GPU (or CPU)
with tf.device('/GPU:0'):  # Or '/CPU:0'
    model4.fit(Xtrain, Ytrain, epochs=15, batch_size=64, validation_data=(Xtest, Ytest))


# Evaluate model4
loss, accuracy = model4.evaluate(Xtest, Ytest)
print(f"Model 4 - Test Loss: {loss}")
print(f"Model 4 - Test Accuracy: {accuracy}")

plot_model(model4)
model4.save('model4.h5')


In [ ]:
plot_model(model4)

In [ ]:
model4.save('forth_model.h5')

In [ ]:
# Faux , faux
# vrai faux
# vrai , vrai
# faux , vrai

import pandas as pd

try:
    data = pd.read_excel("balanced_dataset.xlsx")

    # Create new columns for the four classes
    data['class_1'] = 0  # False, False
    data['class_2'] = 0  # True, False
    data['class_3'] = 0  # True, True
    data['class_4'] = 0  # False, True

    # Assign values to the new classes based on 'Robot_ProtectiveStop' and 'grip_lost'
    data.loc[(data['Robot_ProtectiveStop'] == False) & (data['grip_lost'] == False), 'class_1'] = 1
    data.loc[(data['Robot_ProtectiveStop'] == True) & (data['grip_lost'] == False), 'class_2'] = 1
    data.loc[(data['Robot_ProtectiveStop'] == True) & (data['grip_lost'] == True), 'class_3'] = 1
    data.loc[(data['Robot_ProtectiveStop'] == False) & (data['grip_lost'] == True), 'class_4'] = 1

    # Drop the original combined target columns
    data = data.drop(['Robot_ProtectiveStop', 'grip_lost'], axis=1)

    # Save the modified DataFrame to a new Excel file
    final_db_file = "final_dataset_four_classes.xlsx"
    data.to_excel(final_db_file, index=False)
    print(f"Final database with four classes saved to {final_db_file}")

    from google.colab import files
    files.download(final_db_file)

except FileNotFoundError:
    print("'balanced_dataset.xlsx' not found. Please make sure the file exists in the current directory.")
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Define your target variable (y) as one of the four classes, for example 'class_1'
y = data['class_1']  # Replace with the desired class

# Define your features (X) as all columns except the target class columns
X = data.drop(['class_1', 'class_2', 'class_3', 'class_4'], axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a Decision Tree classifier
clf = DecisionTreeClassifier(random_state=42)  # You can tune hyperparameters here
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Decision Tree Accuracy: {accuracy}")


In [ ]:

from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score

# Assuming y_test and y_pred are already defined from your model's predictions

# Classification Report
print(classification_report(y_test, y_pred))

# Confusion Matrix
print(confusion_matrix(y_test, y_pred))

# Precision
precision = precision_score(y_test, y_pred, average='weighted') #or 'macro', 'micro'
print(f"Precision: {precision}")

# Recall
recall = recall_score(y_test, y_pred, average='weighted') #or 'macro', 'micro'
print(f"Recall: {recall}")

# F1-Score
f1 = f1_score(y_test, y_pred, average='weighted') #or 'macro', 'micro'
print(f"F1-Score: {f1}")


In [ ]:


# Save the trained Decision Tree model to a file
dump(clf, 'decision_tree_model.joblib')
print("Decision Tree model saved to decision_tree_model.joblib")

# Download the saved model file
files.download('decision_tree_model.joblib')


In [ ]:


from sklearn.ensemble import RandomForestClassifier

# Assuming X_train, X_test, y_train, y_test are already defined from previous code

# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust n_estimators

# Train the classifier
rf_classifier.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf_classifier.predict(X_test)

# Evaluate the model (example using accuracy)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy: {accuracy_rf}")

# You can also calculate other metrics like precision, recall, F1-score, etc. as shown in the previous example
print(classification_report(y_test, y_pred_rf))
print(confusion_matrix(y_test, y_pred_rf))


In [ ]:


# Save the trained Random Forest model to a file
dump(rf_classifier, 'random_forest_model.joblib')
print("Random Forest model saved to random_forest_model.joblib")

# Download the saved model file
files.download('random_forest_model.joblib')


In [ ]:

from sklearn.model_selection import GridSearchCV

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'criterion': ['gini', 'entropy'],  # Splitting criterion
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]  # Minimum samples required to be at a leaf node
}

# Create a Decision Tree classifier
clf = DecisionTreeClassifier(random_state=42)

# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring='accuracy') # 5-fold cross-validation
grid_search.fit(X_train, y_train)

# Get the best hyperparameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best Hyperparameters: {best_params}")
print(f"Best Cross-Validation Accuracy: {best_score}")

# Train a new Decision Tree model with the best hyperparameters
best_clf = DecisionTreeClassifier(**best_params, random_state=42)
best_clf.fit(X_train, y_train)

# Evaluate the model with the best hyperparameters
y_pred_best = best_clf.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)
print(f"Accuracy with Best Hyperparameters: {accuracy_best}")

# Print classification report for the best model
print(classification_report(y_test, y_pred_best))
